
### Statistical Classification
We want to classify women working and predict wheter or not a new sample is working. This inference is based the set of features shown below. 

* $x_1$: Husband's income
* $x_2$: Years of education
* $x_3$: Years of work experience
* $x_4$: Age
* $x_5$: Number of children < 6 years
* $x_6$: Number of children > 6 years

The dataset will be split (70/30) into training and testing data. Two classification models will be used: logistic regression (parametric model) and k-nearest-neighbors (non-parametric model).

In [2]:
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt

# Plotting configuration
%config InlineBackend.figure_format = 'retina'
plt.style.use(['science', 'notebook', 'grid'])

In [16]:
df = pd.read_csv('data/WomenAtWork.dat', delimiter='\t', engine='python')

split_index = int(np.ceil(0.7 * len(df)))
df_training = df[0:split_index]
df_test = df[split_index:]

### Logistic Regression

We begin by applying logistic regression. Logistic regression is a transformed version of linear regression, which limits the output to $[0,1]$, which represents a probability of being in a class or not. Note, we are only making inference on one class.

For logistic regression, we transform the $y_i$ using the Sigmoid function defined as: $f(x) \triangleq \frac{1}{1 + e^x}$. Thus we obtain the following model for our classification problem:
$$
\mathbb{P}({y_i}=1| \mathbf{x}, \mathbf{\theta}) = \frac{1}{1 + \exp \{\theta_0 + \theta_1 x_1 + \ldots + \theta_6 x_6\}} = \frac{1}{1 + \exp \{\mathbf{x}^{\text{T}}\mathbf{\theta}\}}
$$

The sample of observations $\{y_i\}_{i=1}^n$ can now be intepreted as a sequence of Bernoulli random variables. Thus we can obtain the following log-likelihood function rather easily:

$$
-\ell(\bold{X}, \bold{y} | \mathbf{\theta}) = \sum_{i=1}^n \big[(y_i-1)\mathbf{\theta}^{\text{T}}\mathbf{x}_i - \ln(1+\exp\{\mathbf{\theta}^{\text{T}}\mathbf{x}_i\})\big]
$$

Next, we fit the model to the training data using the log-likelihood function above. 

### Accuracy & Confusion Matrix

### kNN 

### Accuracy & Confusion Matrix

### Comparison